# Team 6 Project: MINJUSTICIA

In [1]:
import pandas as pd
import numpy as np
import calendar
from matplotlib import pyplot as plt
%matplotlib inline
pd.options.display.max_rows = 15
import requests
import io
import plotly
import plotly.express as px
from datetime import datetime, date
import psutil

Data

In [29]:
url="https://raw.githubusercontent.com/sagilar/ds4a-team6/master/retomintic/Data_UpdateJune13/reincidencia11junio2020_clean.csv"
df_mj = pd.read_csv(url,sep=",",encoding="utf-8")
# df_mj

In [30]:
df_orig = df_mj.copy()
# df_mj.columns

In [31]:
df_mj["FECHA_CAPTURA"] = pd.to_datetime(df_mj["FECHA_CAPTURA"])
df_mj["FECHA_INGRESO"] = pd.to_datetime(df_mj["FECHA_INGRESO"])
df_mj["FECHA_SALIDA"] = pd.to_datetime(df_mj["FECHA_SALIDA"])

df_mj["MES_INGRESO_INT"]=df_mj["FECHA_INGRESO"].dt.strftime('%m')
df_mj.head()

,INTERNOEN,DELITO,TITULO_DELITO,SUBTITULO_DELITO,TENTATIVA,AGRAVADO,CALIFICADO,FECHA_INGRESO,FECHA_SALIDA,FECHA_CAPTURA,...,NIVEL_EDUCATIVO,HIJOS_MENORES,CONDIC_EXPECIONAL,CODIGO_ESTABLECIMIENTO,ESTABLECIMIENTO,DEPTO_ESTABLECIMIENTO,MPIO_ESTABLECIMIENTO,REGIONAL,ESTADO,MES_INGRESO_INT
0,5496C737C624136DB406CFC6744C684C070A3CD8,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,CONTRA LA SEGURIDAD PUBLICA,DE LOS DELITOS DE PELIGRO COMUN O QUE PUEDEN O...,N,N,N,2010-01-01,2011-12-05,2009-12-30,...,CICLO III,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja,01
1,1097561F306939C26DB4AC60377063DA7E2B079F,TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES,CONTRA LA SALUD PUBLICA,DE LAS AFECTACIONES A LA SALUD PUBLICA,N,N,N,2010-01-01,2011-05-12,2009-12-30,...,CICLO I,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja,01
2,CB7033A707ADE673139F8A17CAD26AD54696C135,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,CONTRA LA SEGURIDAD PUBLICA,DE LOS DELITOS DE PELIGRO COMUN O QUE PUEDEN O...,N,N,N,2010-01-01,2013-06-08,2009-12-30,...,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,01
3,CB7033A707ADE673139F8A17CAD26AD54696C135,HURTO,CONTRA EL PATRIMONIO ECONOMICO,DE LA ESTAFA,N,S,S,2010-01-01,2013-06-08,2009-12-30,...,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,01
4,CB7033A707ADE673139F8A17CAD26AD54696C135,UTILIZACION ILEGAL DE UNIFORMES E INSIGNIAS,CONTRA LA SEGURIDAD PUBLICA,DE LOS DELITOS DE PELIGRO COMUN O QUE PUEDEN O...,N,N,N,2010-01-01,2013-06-08,2009-12-30,...,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,01


In [32]:
for column in ['FECHA_INGRESO', 'FECHA_SALIDA', 'FECHA_CAPTURA']:
    df_mj = df_mj.sort_values(['INTERNOEN', column], ascending = False)
    
    df_mj['DIAS' + column[5:]] = -1*df_mj[column].diff().copy()

    df_mj['DIAS' + column[5:]][(df_mj.INTERNOEN != df_mj.INTERNOEN.shift(1)) | (df_mj['DIAS' + column[5:]] == np.timedelta64(0))] = (datetime.today() - pd.to_datetime(df_mj[column]))


<ipython-input-32-fc0f918af65f>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
df_mj['DIAS_CONDENA'] = df_mj['FECHA_SALIDA'] - df_mj['FECHA_INGRESO'] 
df_mj['DIAS_JUDICIALIZACION'] = df_mj['FECHA_INGRESO'] - df_mj['FECHA_CAPTURA']
df_mj['DIAS_LIBRE'] = (df_mj['DIAS_INGRESO'] - df_mj['DIAS_CONDENA'])
df_mj.tail()
# df_mj['DIAS_CAPTURA'].unique().astype('timedelta64[D]')

,INTERNOEN,DELITO,TITULO_DELITO,SUBTITULO_DELITO,TENTATIVA,AGRAVADO,CALIFICADO,FECHA_INGRESO,FECHA_SALIDA,FECHA_CAPTURA,...,MPIO_ESTABLECIMIENTO,REGIONAL,ESTADO,MES_INGRESO_INT,DIAS_INGRESO,DIAS_SALIDA,DIAS_CAPTURA,DIAS_CONDENA,DIAS_JUDICIALIZACION,DIAS_LIBRE
39682,000393AD7FE65FBBE53A61B0EAA1FB9A62F71450,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,CONTRA LA SEGURIDAD PUBLICA,DE LOS DELITOS DE PELIGRO COMUN O QUE PUEDEN O...,N,N,N,2013-05-17,2015-11-08,2010-04-22,...,BOGOTA D.C.,CENTRAL,Baja,05,2634 days 22:46:54.150419,1729 days 22:46:54.418421,3755 days 22:46:54.680457,905 days,1121 days,1729 days 22:46:54.150419
39683,000393AD7FE65FBBE53A61B0EAA1FB9A62F71450,HURTO,CONTRA EL PATRIMONIO ECONOMICO,DE LA ESTAFA,N,S,S,2013-05-17,2015-11-08,2010-04-22,...,BOGOTA D.C.,CENTRAL,Baja,05,2634 days 22:46:54.150419,1729 days 22:46:54.418421,3755 days 22:46:54.680457,905 days,1121 days,1729 days 22:46:54.150419
133508,000357E4F19E3722BA43488F69A7D7DECFC620C4,CONCIERTO PARA DELINQUIR,CONTRA LA SEGURIDAD PUBLICA,DE LOS DELITOS DE PELIGRO COMUN O QUE PUEDEN O...,N,N,N,2019-09-25,NaT,2017-12-14,...,PEREIRA,VIEJO CALDAS,Activo,09,312 days 22:46:54.150419,NaT,962 days 22:46:54.680457,NaT,650 days,NaT
133509,000357E4F19E3722BA43488F69A7D7DECFC620C4,HURTO,CONTRA EL PATRIMONIO ECONOMICO,DE LA ESTAFA,N,S,S,2019-09-25,NaT,2017-12-14,...,PEREIRA,VIEJO CALDAS,Activo,09,312 days 22:46:54.150419,NaT,962 days 22:46:54.680457,NaT,650 days,NaT
112953,000334541B6B3305CE91761942D7B381248777EA,TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES,CONTRA LA SALUD PUBLICA,DE LAS AFECTACIONES A LA SALUD PUBLICA,N,N,N,2018-06-08,NaT,2018-02-08,...,PITALITO,CENTRAL,Activo,06,786 days 22:46:54.150419,NaT,906 days 22:46:54.680457,NaT,120 days,NaT


individuals

In [34]:
individuals = df_mj["INTERNOEN"].unique()
print("length of complete df: " + str(df_mj["INTERNOEN"].count()))
print("length of unique individuals: "+ str(len(individuals)))

length of complete df: 142899
length of unique individuals: 78841


In [35]:
print("number of occurrences by individual")
occurrences = df_mj["INTERNOEN"].value_counts()
occurrences

number of occurrences by individual


71D8863489582B956956B3E244CF97F629E190F8    16
85F84CC906246E467ACC8D3CD73B576E0B264556    15
C6A8D6B8A3433007BDDE0B6A682268EFD0FA16AD    14
5F6A19F239D3786924DB4FEAC108F33219BFD205    14
D46A8E8ACCCCBB5CF90A9A81047E36D2207F25FB    13
                                            ..
EF6290F52FABEE4B9B3FC80DA099D7C3CFC42AAD     1
21D268D13DEDCDF019A44CF4B8343DB9CC2A57DA     1
C43984962FD09290AAAEDD51A5639ADD4A7BCA0F     1
D082ED87AF4888421DC9CD6D8B1A58899AC5D623     1
3C590DA50C2C4FD7A610C4C76512A1097016DED3     1
Name: INTERNOEN, Length: 78841, dtype: int64

In [36]:
for item in occurrences.index:
    print("crimes of individual " + str(item) + ": ")
    crime_list_per_ind = df_mj["DELITO"].where(df_mj["INTERNOEN"]==item)
    for crime in crime_list_per_ind.dropna():
        print(crime + ", ", end="")
    #breaking to check just first ind
    break

crimes of individual 71D8863489582B956956B3E244CF97F629E190F8: 
CONCIERTO PARA DELINQUIR, CONSTREÑIMIENTO ILEGAL, DESPLAZAMIENTO FORZADO, DESTRUCCION Y APROPIACION DE BIENES PROTEGIDOS, FABRICACION  TRAFICO Y PORTE DE ARMAS Y MUNICIONES DE USO PRIVATIVO DE LAS FUERZAS ARMADAS, HOMICIDIO, HOMICIDIO EN PERSONA PROTEGIDA, HURTO, RECLUTAMIENTO ILICITO, SECUESTRO SIMPLE, TORTURA EN PERSONA PROTEGIDA, UTILIZACION ILEGAL DE UNIFORMES E INSIGNIAS, CONCIERTO PARA DELINQUIR, DESPLAZAMIENTO FORZADO, HOMICIDIO, HURTO, 

Ubicación geográfica

In [37]:
dept_count = df_mj["DEPTO_ESTABLECIMIENTO"].value_counts()
dept_count

BOGOTA D.C.                 19008
ANTIOQUIA                   18591
VALLE DEL CAUCA             17346
SANTANDER                    9274
META                         7996
                            ...  
ARAUCA                        513
LA GUAJIRA                    456
PUTUMAYO                      378
AMAZONAS                      349
SAN ANDRES Y PROVIDENCIA      252
Name: DEPTO_ESTABLECIMIENTO, Length: 29, dtype: int64

In [38]:
gender_count = df_mj["GENERO"].value_counts()
gender_count

MASCULINO    131485
FEMENINO      11414
Name: GENERO, dtype: int64

In [39]:
education_count =df_mj["NIVEL_EDUCATIVO"].value_counts()
print(education_count)

df_mj["NIVEL_EDUCATIVO"].describe()

CICLO II               54353
CICLO I                28198
CICLO III              27839
CICLO IV               25109
ANALFABETA              3717
PROFESIONAL             1636
TECNICO                 1328
TECNOLOGICO              460
ESPECIALIZACION          145
TECNICO PROFESIONAL       61
POST GRADO                33
MAGISTER                  20
Name: NIVEL_EDUCATIVO, dtype: int64


count       142899
unique          12
top       CICLO II
freq         54353
Name: NIVEL_EDUCATIVO, dtype: object

In [40]:
education_male = df_mj["NIVEL_EDUCATIVO"][df_mj["GENERO"]=="MASCULINO"].value_counts()
education_female = df_mj["NIVEL_EDUCATIVO"][df_mj["GENERO"]=="FEMENINO"].value_counts()
print("Male education leves\n",education_male,"\n\n","Female education leves\n", education_female)

Male education leves
 CICLO II               49994
CICLO III              25844
CICLO I                25790
CICLO IV               23352
ANALFABETA              3389
PROFESIONAL             1375
TECNICO                 1128
TECNOLOGICO              397
ESPECIALIZACION          115
TECNICO PROFESIONAL       51
POST GRADO                30
MAGISTER                  20
Name: NIVEL_EDUCATIVO, dtype: int64 

 Female education leves
 CICLO II               4359
CICLO I                2408
CICLO III              1995
CICLO IV               1757
ANALFABETA              328
PROFESIONAL             261
TECNICO                 200
TECNOLOGICO              63
ESPECIALIZACION          30
TECNICO PROFESIONAL      10
POST GRADO                3
Name: NIVEL_EDUCATIVO, dtype: int64


In [41]:
df_mj["TITULO_DELITO"].value_counts()

CONTRA EL PATRIMONIO ECONOMICO                 56860
CONTRA LA SEGURIDAD PUBLICA                    31591
CONTRA LA SALUD PUBLICA                        25028
CONTRA LA VIDA Y LA INTEGRIDAD PERSONAL        11329
CONTRA LA FAMILIA                               3689
                                               ...  
DELITOS DE PARAMILITARISMO Y SICARIATO             9
DELITOS CONTRA LA DISCIPLINA                       5
ACCION DE TUTELA                                   4
DELITOS CONTRA LA ADMINISTRACION PUBLICA           2
CONTRA LA EXISTENCIA Y SEGURIDAD DEL ESTADO        1
Name: TITULO_DELITO, Length: 25, dtype: int64

In [42]:
df_genero = pd.DataFrame(df_mj.groupby(["GENERO","TITULO_DELITO"])["INTERNOEN"].agg('count'))
df_genero.reset_index(inplace=True)

df_geo_gen = pd.DataFrame(df_mj.groupby(["GENERO","DEPTO_ESTABLECIMIENTO"])["INTERNOEN"].agg('count'))
df_geo_gen.reset_index(inplace=True)

df_crimen = pd.DataFrame(df_mj.groupby(["TITULO_DELITO"])["INTERNOEN"].agg('count'))
df_crimen.reset_index(inplace=True)

df_edu_gen = pd.DataFrame(df_mj.groupby(["GENERO","NIVEL_EDUCATIVO"])["INTERNOEN"].agg('count'))
df_edu_gen.reset_index(inplace=True)

In [43]:

fig = px.bar(df_edu_gen, x='NIVEL_EDUCATIVO', y='INTERNOEN',
             hover_data=['GENERO','NIVEL_EDUCATIVO'], color='GENERO',
             labels={'INTERNOEN':'Frecuencia del crimen','NIVEL_EDUCATIVO':'Nivel educativo'}, 
             height=500, barmode='group', text="INTERNOEN")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='hide',
                  title="Crimenes por nivel educativo",
                 )
fig.show()
fig.write_image("fig3.png")

In [19]:
fig = px.bar(df_genero[df_genero["GENERO"]=="MASCULINO"], x='TITULO_DELITO', y='INTERNOEN',
             hover_data=['TITULO_DELITO'], color='TITULO_DELITO',
             labels={'INTERNOEN':'Amount of recidivist crimes'}, height=400, barmode='stack', text="INTERNOEN")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='hide', xaxis_tickangle=-45, showlegend=False, autosize=False,
                 width=800, height=700, margin=dict(l=50, r=50, b=100, t=100, pad=4 ),
                  title="Most commited crimes by men",
                 )
fig.update_xaxes( tickfont=dict(family='Rockwell', color='black', size=8)) #tickangle=45,

fig.show()


In [20]:
fig = px.bar(df_genero[df_genero["GENERO"]=="FEMENINO"], x='TITULO_DELITO', y='INTERNOEN',
             hover_data=['TITULO_DELITO'], color='TITULO_DELITO',
             labels={'INTERNOEN':'Amount of recidivist crimes'}, height=400, barmode='stack', text="INTERNOEN")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='hide', xaxis_tickangle=-45, showlegend=False, autosize=False,
                 width=800, height=700, margin=dict(l=50, r=50, b=100, t=100, pad=4 ),
                  title="Most commited crimes by women",
                 )
fig.update_xaxes( tickfont=dict(family='Rockwell', color='black', size=8)) #tickangle=45,

fig.show()

In [26]:
fig = px.bar(df_genero, x='TITULO_DELITO', y='INTERNOEN',
             hover_data=["GENERO",'TITULO_DELITO'], color='GENERO',
             labels={'INTERNOEN':'Frecuencia del delito',"TITULO_DELITO":"Delito"}, height=400, barmode='relative', text="INTERNOEN")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=7, uniformtext_mode='hide', xaxis_tickangle=-45, showlegend=True, autosize=False,
                 width=800, height=700, margin=dict(l=50, r=50, b=100, t=100, pad=4 ),
                  title="Crimenes más cometidos por género",
                 )
fig.update_xaxes( tickfont=dict(family='Rockwell', color='black', size=8)) #tickangle=45,

fig.show()

fig.write_image("fig1.png")

In [27]:
fig = px.bar(df_geo_gen, x='DEPTO_ESTABLECIMIENTO', y='INTERNOEN',
             hover_data=["GENERO",'DEPTO_ESTABLECIMIENTO'], color='GENERO',
             labels={'INTERNOEN':'Frecuencia del delito',"DEPTO_ESTABLECIMIENTO":"Departamento"},
             barmode='relative', text="INTERNOEN")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='hide', xaxis_tickangle=-45, showlegend=True, autosize=False,
                 width=900, height=700, margin=dict(l=50, r=50, b=100, t=100, pad=4 ),
                  title="Distribucion geografica de los delitos",
                 )
fig.update_xaxes( tickfont=dict(family='Rockwell', color='black', size=8)) #tickangle=45,

fig.show()

fig.write_image("fig2.png")

In [45]:
economic_crime = df_mj[(df_mj["TITULO_DELITO"]=="CONTRA EL PATRIMONIO ECONOMICO")].groupby(["GENERO", "DELITO"])["DELITO"].agg("count")
economic_crime = pd.DataFrame(economic_crime)
economic_crime.rename(columns={"DELITO": "COUNT"},inplace=True)
economic_crime.reset_index(inplace=True)

In [25]:
fig = px.bar(economic_crime, x='DELITO', y='COUNT',
             hover_data=["GENERO",'DELITO'], color='GENERO',
             labels={'COUNT':'Amount of recidivist economic crimes'}, barmode='relative', text="COUNT")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='hide', xaxis_tickangle=-45, showlegend=True, autosize=False,
                 width=900, height=700, margin=dict(l=50, r=50, b=100, t=100, pad=4 ),
                  title="Most commited crimes against economic patrimony per gender",
                 )
fig.update_xaxes( tickfont=dict(family='Rockwell', color='black', size=8)) #tickangle=45,

fig.show()

In [46]:
female_crime = df_mj[(df_mj["TITULO_DELITO"]=="CONTRA LA SALUD PUBLICA")].groupby(["GENERO", "DELITO"])["DELITO"].agg("count")
female_crime = pd.DataFrame(female_crime)
female_crime.rename(columns={"DELITO": "COUNT"},inplace=True)
female_crime.reset_index(inplace=True)


In [47]:
fig = px.bar(female_crime, x='DELITO', y='COUNT',
             hover_data=["GENERO",'DELITO'], color='GENERO',
             labels={'COUNT':'Amount of recidivist crimes against public health'}, barmode='relative', text="COUNT")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='hide', xaxis_tickangle=-45, showlegend=True, autosize=False,
                 width=900, height=700, margin=dict(l=50, r=50, b=100, t=100, pad=4 ),
                  title="Most commited crimes against public health per gender",
                 )
fig.update_xaxes( tickfont=dict(family='Rockwell', color='black', size=8)) #tickangle=45,

fig.show()

In [50]:
df_mj[(df_mj["TITULO_DELITO"]=="CONTRA LA SALUD PUBLICA") & (df_mj["GENERO"]=="MASCULINO")]["DELITO"].value_counts()[0:5]


TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES                       19610
DESTINACION ILICITA DE MUEBLES O INMUEBLES                             388
TRAFICO DE SUSTANCIAS PARA PROCESAMIENTO DE NARCOTICOS                 252
CONSERVACION O FINANCIACION DE PLANTACIONES                             81
CORRUPCION DE ALIMENTOS PRODUCTOS MEDICOS O MATERIAL PROFILACTICO       63
Name: DELITO, dtype: int64

In [49]:
df_mj[(df_mj["TITULO_DELITO"]=="CONTRA LA VIDA Y LA INTEGRIDAD PERSONAL") & (df_mj["GENERO"]=="MASCULINO")]["DELITO"].value_counts()[0:5]


HOMICIDIO                      8625
LESIONES PERSONALES            1995
FEMINICIDIO                      95
HOMICIDIO PRETERINTENCIONAL      54
HOMICIDIO CULPOSO                48
Name: DELITO, dtype: int64

In [48]:
df_mj[(df_mj["TITULO_DELITO"]=="CONTRA LA SEGURIDAD PUBLICA") & (df_mj["GENERO"]=="MASCULINO")]["DELITO"].value_counts()[0:5]

FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO O MUNICIONES                                                                    16445
CONCIERTO PARA DELINQUIR                                                                                                       8709
FABRICACIÓN, TRÁFICO, PORTE O TENENCIA DE ARMAS DE FUEGO, ACCESORIOS, PARTES O MUNICIONES                                      2149
FABRICACION  TRAFICO Y PORTE DE ARMAS Y MUNICIONES DE USO PRIVATIVO DE LAS FUERZAS ARMADAS                                     1748
FABRICACIÓN, TRÁFICO Y PORTE DE ARMAS, MUNICIONES DE USO RESTRINGIDO, DE USO PRIVATIVO DE LAS FUERZAS ARMADAS O EXPLOSIVOS      360
Name: DELITO, dtype: int64

In [51]:
df_tent = df_mj[(df_mj.AGRAVADO == "S")] # & (df_mj.AGRAVADO == "S") & (df_mj.CALIFICADO == "S")
df_tent.groupby(["DELITO"])["INTERNOEN"].count()

DELITO
ABIGEATO                                                                      2
ACCESO ABUSIVO A UN SISTEMA INFORMATICO                                       2
ACCESO CARNAL ABUSIVO CON MENOR DE CATORCE AÑOS                             123
ACCESO CARNAL O ACTO SEXUAL ABUSIVOS CON INCAPAZ DE RESISTIR                 11
ACCESO CARNAL O ACTO SEXUAL EN PERSONA PUESTA EN INCAPACIDAD DE RESISTIR      4
                                                                           ... 
USO DE MENORES DE EDAD PARA LA COMISION DE DELITOS                           14
UTILIZACION ILEGAL DE UNIFORMES E INSIGNIAS                                   1
VIOLACION DE DATOS PERSONALES                                                 9
VIOLENCIA CONTRA SERVIDOR PUBLICO                                             1
VIOLENCIA INTRAFAMILIAR                                                     434
Name: INTERNOEN, Length: 85, dtype: int64

In [11]:
df_mj[df_mj.GENERO == "MASCULINO"].shape[0]/df_mj.shape[0]

0.9201254032568458